In [1]:
import time
import numpy as np

In [3]:
def func1(i: int) -> int:
    time.sleep(0.001)
    return i + 1

def func2(i: int) -> int:
    time.sleep(0.001)
    return i * 2

def func3(i: int) -> int:
    time.sleep(0.001)
    return i % 3

In [4]:
# Serial Run
serial = np.arange(0,10000)
print(len(serial))
start = time.time()
 
serial = np.array([func1(xi) for xi in serial])
print(serial)
serial = np.array([func2(xi) for xi in serial])
print(serial)
serial = np.array([func3(xi) for xi in serial])
print(serial)

    
print("total ingestion time: {%.4f}s" % float(time.time() - start))

10000
[    1     2     3 ...  9998  9999 10000]
[    2     4     6 ... 19996 19998 20000]
[2 1 0 ... 1 0 2]
total ingestion time: {31.8856}s


In [5]:
import ray
ray.__version__

'1.10.0'

In [6]:
# Connect to a cluster
ray.init("ray://10.100.3.60:10001")

ClientContext(dashboard_url='192.168.95.118:8265', python_version='3.7.7', ray_version='1.10.0', ray_commit='5ea565317a8104c04ae7892bb9bb41c6d72f12df', protocol_version='2021-12-07', _num_clients=1, _context_to_restore=<ray.util.client._ClientContext object at 0x7f693b257350>)

In [7]:
# Create a dataset and then create a pipeline from it.
base = ray.data.range(10000)
print(base)
# -> Dataset(num_blocks=200, num_rows=1000000, schema=<class 'int'>)
pipe = base.window(blocks_per_window=50)
print(pipe)
# -> DatasetPipeline(num_windows=20, num_stages=1)
#pipe.show_windows()

Dataset(num_blocks=200, num_rows=10000, schema=<class 'int'>)
DatasetPipeline(num_windows=4, num_stages=1)


In [8]:
# Applying transforms to pipelines adds more pipeline stages.
pipe = pipe.map(func1)
pipe = pipe.map(func2)
pipe = pipe.map(func3)
print(pipe)
# -> DatasetPipeline(num_windows=20, num_stages=4)

DatasetPipeline(num_windows=4, num_stages=4)


In [9]:
start = time.time()
pipe.show_windows()
print("total ingestion time: {%.4f}s" % float(time.time() - start))

Stage 1:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Stage 2:   0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


Stage 0:  50%|█████     | 2/4 [00:02<00:02,  1.33s/it]

Stage 0:  75%|███████▌  | 3/4 [00:04<00:01,  1.41s/it]


Stage 3:  25%|██▌       | 1/4 [00:04<00:13,  4.56s/it]

------ Epoch 0 ------
=== Window 0 ===
2
1
0
2
1
0
2
1
0
2




Stage 0: 100%|██████████| 4/4 [00:05<00:00,  1.48s/it]


Stage 3:  50%|█████     | 2/4 [00:06<00:05,  2.93s/it]

=== Window 1 ===
1
0
2
1
0
2
1
0
2
1




Stage 1: 100%|██████████| 4/4 [00:07<00:00,  1.73s/it]


Stage 3:  75%|███████▌  | 3/4 [00:07<00:02,  2.31s/it]

=== Window 2 ===
0
2
1
0
2
1
0
2
1
0




Stage 2: 100%|██████████| 4/4 [00:08<00:00,  1.77s/it]


Stage 0: 100%|██████████| 4/4 [00:09<00:00,  2.25s/it]

=== Window 3 ===
2
1
0
2
1
0
2
1
0
2
total ingestion time: {9.0432}s


In [10]:
# Connect to a cluster
ray.shutdown()